<a target="_blank" href="https://colab.research.google.com/github/JLDC/Data-Science-Fundamentals/blob/master/notebooks/205-my-own-neural-network-1.ipynb">
    <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Open this notebook in Google Colab
</a>

___

# Building our own Neural Network (pt. 1)
___
In this script we will build our own *home-made* neural network without the help of the `scikit-learn` package's model. We will train our network using **backpropagation** and **gradient descent**, as is customary in neural networks. Backpropagation and gradient descent are closely related, but there is an important difference:  
+ **Gradient descent** is the optimization algorithm used to *minimize the loss*.
+ **Backpropagation** is the differentiation algorithm used to *compute the gradients*.

Going through this notebook will give you a better understanding of what these fancy graphical representations of neural networks really mean in terms of math and programming. For simplicity, we omit any bias terms, i.e., constant-value features in the input or hidden layers.

We will look at a classification example with only two input features, a single hidden layer with $n_Z$ hidden nodes, and a single output. The code is flexible with respect to the number of hidden nodes, so you can adjust them to see how the results change with more or less nodes in the hidden layer. When training the network, we will actually use stochastic gradient descent rather than gradient descent. Stochastic gradient descent is a special case of gradient descent. The idea is to compute the gradient using only a random subsample of the data instead of the full dataset. This has three main advantages:
1. It is typically quicker to reach a *good* result.
2. It is less likely that we end up getting stuck in a local minimum.
3. It slightly simplifies the code.

As you will see, neural networks entail a lot of matrix multiplications, so hopefully your linear algebra is up-to-date!

Note that this way of building a neural network is purely pedagogical. There is no chance that you would actually build your own neural network in a practical scenario. You would simply implement a network using one of the many deep learning packages in Python (see the first notebook *Introduction to Python*, for links to some of these packages).

___
## Data pre-processing

In [ ]:
# Import necessary packages
import numpy as np # Numerical computation package
import pandas as pd # Dataframe package
import matplotlib.pyplot as plt # Plotting package
import matplotlib as mpl # For colormaps
np.random.seed(1) # Set the random seed for reproduceability

# Define the path where the data is stored
DATA_PATH = "https://raw.githubusercontent.com/JLDC/Data-Science-Fundamentals/master/data"

Once again, will work with the WDBC dataset. There is nothing new here, this is very similar to what we did in the notebook on perceptrons.

In [ ]:
# Read in the WDBC dataset
wdbc = pd.read_csv(f"{DATA_PATH}/wdbc.csv")
# Keep only necessary columns: the diagnosis, the perimeter, and the severity of concave portions
# of the cell nucleus
wdbc = wdbc[["perimeterM", "concaveM", "diagnosis"]]
wdbc # Display the dataset

### 🙀 🤯 Standardization

First, we will begin by standardizing the data. **This is an absolute must for neural networks**, even when working with a library instead of your own *home-built* network, standardizing your data is key!

But why is it so important to standardize with neural networks as opposed to other statistical models? The answer lies in **gradient descent**. In truth, it's not some neural network specific architecture that makes it so important to standardize but simply the application of gradient descent. To see why, consider the following example.

Say that we have two features, $\mathbf{x}_1$ and $\mathbf{x}_2$, and we are trying to find the weights $w_1$, $w_2$ which minimize the mean squared error $\frac{1}{N} \sum_{i=1}^N (y^{(i)} - w_1 x_1^{(i)} - w_2 x_2^{(i)})^2$. So just as we did in some previous notebooks. Recall, that the weight updates are then given by:

\begin{align}
w_1 &\leftarrow w_1 - \eta \frac{\partial \text{MSE}}{\partial w_1} \\
w_2 &\leftarrow w_2 - \eta \frac{\partial \text{MSE}}{\partial w_2} 
\end{align}

which is equivalent to

\begin{align}
w_1 &\leftarrow w_1 - \eta \frac{1}{N} \sum_{i=1}^N (y^{(i)} - w_1 x_1^{(i)} - w_2 x_2^{(i)}) (-x_1^{(i)}) \\
w_2 &\leftarrow w_2 - \eta \frac{1}{N} \sum_{i=1}^N (y^{(i)} - w_1 x_1^{(i)} - w_2 x_2^{(i)}) (-x_2^{(i)}).
\end{align}

The important part of this equation is the last one. The updates on the weight $w_1$ are **scaled** by the values of the feature $x_1$, and the same happens for the weights $w_2$ with the features $x_2$.

Consider the iris example, where $x_1$ was the sepal width in centimeters, and $x_2$ the sepal length in centimeters. Both values in centimeters are on a somewhat similar scale, so this is not a problem. Consider, however, what would happen if instead we were to use the the sepal width in nanometers? Suddenly, our $x_1$ would be on average around 10 million times larger than the $x_2$ and the weight updates of $w_1$ would completely dominate the learning rate! So in the end, we would be dealing with a learning procedure where either $w_1$ has crazy explosive updates or $w_2$ nearly doesn't move, this would make it very hard for our network to converge!

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Create our features and label data
X = np.array(wdbc[["perimeterM", "concaveM"]])
y = np.array(wdbc["diagnosis"] == "M")

In [ ]:
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Quick verification that our mean and std are (0, 1)
print(X.mean(axis=0).round(5))
print(X.std(axis=0))

___
## Training the neural network

There are a few parameters we need to define before we can start with training the network:

+ $N_X = K$, the number of nodes in the input layer (number of features we have)
+ $N_Z$, the number of nodes in the hidden layer.
+ The number of **epochs**. Recall that with gradient descent, we have to specify *how long* we want our procedure to repeat. This is encoded in the number of epochs.
+ $\eta$, the learning rate, a small number which governs *how large* our weight updates will be at each update step

In [ ]:
# Initialize some parameters
N, K = X.shape # Number of observations and features
NZ = 3 # Number of hidden nodes
epochs = 100 # Number of training epochs
eta = 0.01 # Learning rate
init_range = [-.5, .5] # The range of our uniform distribution for weight initialization

___
### 🙀 🤯 Mathematical formulation

Before we go ahead and write the code, it's a good idea to review how a neural network is supposed to work from a mathematical perspective. First, let's go over the individual pieces of our network:  
+ $N$, the number of observations ($N=569$) in our case
+ $K$, the number of features ($K=2$) in our case
+ $N_Z$, the number of nodes in the hidden layer
+ $\mathbf{X}$, a $N \times K$ matrix of data inputs
+ $\mathbf{y}$, a column vector of targets with $N$ elements
+ $\mathbf{W}_1$, a $K \times N_Z$ a weight matrix which *passes* the input data to the hidden layer (before activation)
+ $g_1: \mathbb{R} \to \mathbb{R}$, an activation function, applied element-wise to the output of the first layer
+ $\mathbf{W}_2$, a $N_Z \times 1$ a weight matrix (or a column vector) which *passes* the data from hidden layer to the output layer (before activation)
+ $g_2: \mathbb{R} \to \mathbb{R}$, an activation function, applied element-wise to the output of the second layer

Putting all these elements together, the neural network $f : \mathbb{R}^{N \times K} \to \mathbb{R}^N$ can be expressed as:

$$f(\mathbf{X}) = g_2(g_1(\mathbf{X} \mathbf{W}_1)\mathbf{W}_2)$$

Can you recall what we are training in this neural network? That's right, the weights $\mathbf{W}_1$ and $\mathbf{W}_2$. That's all, everything else is *given* or *defined*. Of course, to train our network, we need to start with some values for $\mathbf{W}_1$ and $\mathbf{W}_2$. We will do so by randomly initializing each element in $\text{unif.}[-0.5, 0.5]$.
___



#### 🤔 Pause and ponder
Why is $\mathbf{W}_1$ a $K \times N_Z$ matrix and $\mathbf{W}_2$ a $N_Z \times 1$ matrix? Could we have chosen other dimensions? 

If $\mathbf{A}$ is $P \times Q$ and $\mathbf{B}$ is $Q \times R$, can we compute $\mathbf{AB}$, and if so, what is its dimension? What about $\mathbf{BA}$ ?
___

In [ ]:
# Initialize weight matrices
np.random.seed(72) # Set seed
W1 = np.random.rand(K, NZ) # Randomly initialize W1
W1 = W1 * (init_range[1] - init_range[0]) + init_range[0] # Constrain to range
W2 = np.random.rand(NZ, 1) # Randomly initialize W2
W2 = W2 * (init_range[1] - init_range[0]) + init_range[0] # Constrain to range

___
#### 🤔 Pause and ponder
Do you see how the above weights relate to the graphical representation seen in class?
___

In [ ]:
# We will use the sigmoid function as the activation function
sigmoid = lambda x: 1 / (1 + np.exp(-x))

Because we will use it a few times, we define a function for the **forward pass** of our network. Recall the mathematical formulation above, you will see that this function is nothing else but

$$f(\mathbf{X}) = g_2(g_1(\mathbf{X} \mathbf{W}_1)\mathbf{W}_2)$$

In [ ]:
# Forward pass helper
def forward_pass(X, W1, W2):
    # Compute the matrix multiplication of the input layer
    S = X @ W1
    
    # Pass through the non-linear activation function
    Z = sigmoid(S)
    
    # Compute the matrix multiplication of the hidden layer
    T = Z @ W2
    
    # Pass through the non-linear activation function 
    # (This should always be sigmoid for the output to be (0, 1))
    return sigmoid(T).flatten() # Transform the output back to a vector instead of Nx1 matrix

In [ ]:
# This function, we will use to evaluate our predictions
# ⚠️ Look at the details of this function only after you have gone through the
# ⚠️ code with the training loop below. See the questions at the end of the script
def eval_predictions(X, y, W1, W2):
    # Use the forward_pass function defined above to compute our estimated probability
    prob = forward_pass(X, W1, W2)
    
    # To avoid log(0), clip the probability to not be exactly zero or one
    prob = np.clip(prob, 1e-8, 1-1e-8)
    # Calculate the loss function (negative log-likelihood in this case)
    loss = - np.sum(y * np.log(prob) + (1 - y) * np.log(1 - prob))

    # Convert probabilities into (0, 1) prediction
    pred = (prob >= 0.5).astype(int)
    
    # Compute number of misclassification
    misclassifications = np.mean(pred != y)
    
    # Output results as a dictionary
    return {
        "loss": loss, 
        "misclassifications": misclassifications, 
        "prob": prob, 
        "pred": pred
    }

In [ ]:
# Initializitation of lists for bookkeeping
loss_list = []
misclassification_list = []

# Create an array of indices (which we will shuffle later on) through which we 
# will iterate. This represent the index of the observations
indices = np.arange(N)

In [ ]:
# Ignore this, we use it to print nicely without creating too many lines
from sys import stdout

In [ ]:
np.random.seed(72) # Reset random seed for reproduceability
# Compute the loss and misclassifications BEFORE training
res = eval_predictions(X, y, W1, W2)

# Append to our result lists
loss_list.append(res["loss"])
misclassification_list.append(res["misclassifications"])

# Run the full training loop (iterate over the number of training epochs)
for epoch in range(epochs):
    # Reshuffle the indices
    np.random.shuffle(indices)
    
    # Iterate through each single data point
    for i in indices:
        # Note that we use [i:i+1, :] instead of [i, :] to keep it as a 1x2 matrix
        Xi = X[i:i+1, :] # Extract features for ith observation,
        yi = y[i] # Extract label for ith observation
        
        # ----- Forward pass -----
        # Computes the predictions using Xi, W1, W2. Here we avoid using the
        # forward_pass function because we need the hidden nodes values Zi
        # for backpropagation. So, instead, we repeat the code (ugh...)
        
        # Pass to the hidden nodes (pre-activation)
        Si = Xi @ W1
        # Compute activation function
        Zi = sigmoid(Si)
        
        # Pass to the output nodes
        Ti = Zi @ W2
        # Compute sigmoid probability transformation
        prob_i = sigmoid(Ti)
        
        # Note that, while we would predict 1 if prob_i ≥ .5 and 0 otherwise,
        # for training, it is better to use the raw probability to update
        # the weights in the backward pass. Think of our discussion as to why
        # perceptron is not a good learning algorithm!
        
        # ----- Backward pass -----
        
        error_i = yi - prob_i
        
        # Compute the gradient w.r.t. W2. ⚠️ W2 is a vector! See math derivations
        grad2_i = -error_i * Zi.T
        # Compute the gradient w.r.t. W1. ⚠️ W1 is matrix! Making things even worse
        grad1_i = -error_i * Xi.T @ (W2.T * Zi * (1 - Zi))
        
        # Updating: Move 'eta' units in the direction of the negative gradient
        W1 -= eta * grad1_i
        W2 -= eta * grad2_i
    
    # Evaluate the learning process and store the results into our lists
    res = eval_predictions(X, y, W1, W2)
    loss_list.append(res["loss"])
    misclassification_list.append(res["misclassifications"])    
        
    # Print the current status (🙀 🤯 ignore this part!)
    bar = "".join(["#" if epoch >= t * (epochs // 50) else " " for t in range(50)])
    stdout.write(f"\rEpoch: {epoch+1:>{int(np.floor(np.log10(epochs))+1)}}/{epochs} [{bar}]")

___
## Evaluating the results
That's it, we can go ahead and look at the results of our own custom neural network!

In [ ]:
# Set up the canvas
fig, axs = plt.subplots(1, 2, figsize=(16, 8))
# Plot the loss over the epochs (epoch 0 is before training!)
axs[0].plot(range(len(loss_list)), loss_list)
# Plot the misclassification rate over the epochs
axs[1].plot(range(len(misclassification_list)), misclassification_list)
# Add title, grid, axis labels
for ax in axs:
    ax.grid(True)
    ax.set_xlabel("Epoch number")
axs[0].set_ylabel("Loss (negative log-likelihood)")
axs[0].set_title("Evolution of loss function over training epochs")
axs[1].set_ylabel("Misclassification rate")
axs[1].set_title("Evolution of missclassification rate over training epochs")

Pretty neat! Our *home-built* neural network works as expected! It achieves a 90% correct classification rate in approximately 20 epochs and then the performance starts stagnating at around 40 epochs. Of course, as you well know, we shouldn't read too much into this, this is purely in-sample, but independent of the out-of-sample performance, our machinery works, which is the main takeaway.

We can also have a look at the weights to see what the final weights are (we use `pandas` for prettier table view).

In [ ]:
# Display the weight matrix from the input layer to the hidden layer
pd.DataFrame(W1, columns=[f"Z{i}" for i in range(NZ)], index=["X1", "X2"])

In [ ]:
# Display the weight matrix  from the hidden layer to the output layer
pd.DataFrame(W2, columns=["T"], index=[f"Z{i}" for i in range(NZ)])

___
## Visualizing the decision regions

Since our feature space is two dimensional, it's easy to *visualize the decision regions* of our neural network. Think about it: every data point can be represented on an $(x_1, x_2)$ plane, and, for every data point, we can make a prediction using our neural network. Combining this, we can create a grid of thinly spaced $(x_1, x_2)$ points, feed them to our neural network and plot them according to the output it suggests.

This is nice because it helps us understand the underlying relationship that was uncovered by our neural network. Recall how, during the perceptron class, we drew the perceptron separation line on the features plane. Do you think the neural network will also split the feature space using a line? Why, why not?

In [ ]:
# Define granularity and limits of grids
npoints = 200
x1_lims = (X[:, 0].min() - .1, X[:, 0].max() + .1)
x2_lims = (X[:, 1].min() - .1, X[:, 1].max() + .1)

# Create the x1 and x2 arrays
x1 = np.linspace(*x1_lims, num=npoints)
x2 = np.linspace(*x2_lims, num=npoints)
preds = np.empty((npoints, npoints))

# Compute the predictions of our network for every single combination
for i in range(x2.shape[0]):
    for j in range(x1.shape[0]):
        # Create the 1x2 input
        xi = np.array([[x1[j], x2[i]]])
        # Compute the prediction of the network
        preds[i, j] = forward_pass(xi, W1, W2)

# Small trick to scale back values to their value before standardization
Xt = scaler.inverse_transform(np.array([x1, x2]).T)
x1, x2 = Xt[:, 0], Xt[:, 1]

In [ ]:
# Set up the canvas
fig, ax = plt.subplots(figsize=(12, 8))
# Add decision regions
ax.contourf(x1, x2, preds, cmap=mpl.colormaps["coolwarm"], alpha=0.8)

# Add true data points
for diag in ["B", "M"]:
    subset = wdbc.loc[wdbc["diagnosis"] == diag]
    ax.scatter(subset["perimeterM"], subset["concaveM"], label=diag, alpha=0.9)
    
# Add grid, labels, etc.
ax.grid(True)
ax.set_xlabel("Perimeter")
ax.set_ylabel("Concavity")
ax.legend()

The benign tumors are shown in blue above and the malignant ones in orange. The color in the background are the decision regions of our neural network. We notice that the decision boundary is a straight line. The intensity of the color regions show *how sure* the neural network is in making the prediction, i.e. in the dark purple area in the top right, the network assigns a very high probability that the tumor is malignant while in the lower left, the probability is very low. Around the white band however, the probability is around 0.5. Of course, in the end, a tumor is always classified as either benign or malignant, but this helps us visualize where the predictions are more *unstable*.

___
### Exercises

#### <font style="color:green">**➡️ ✏️ Question 1**</font>
Try playing around with the parameters. Change the hidden nodes, learning rate, number of epochs.

How do the results change? Do they get better or worse? How do the decision region change?

*Hint*: If you encounter an error, try changing some parameters back. A too high learning rate can cause your weights to explode and `numpy` won't be able to handle such large numbers (in particular when trying to exponentiate them!)

#### <font style="color:green">**➡️ ✏️ Question 2**</font>

Carefully inspect the code of `eval_predictions(...)` and `forward_pass(...)`.

+ Why do we compute the `forward_pass` manually again in the training loop? Is there a difference?
+ Do we need `eval_predictions` to train our network?

Discuss with your classmates and try to explain the code to each other!

#### <font style="color:green">**➡️ ✏️ Question 3 (optional)**</font>

Read the *advanced* part of the class notes on neural networks and try to implement the contents into the code. 

*Hint*: The main difference is that there are two separate *models* for each of the two output layers. This means that we treat the output layer in the same way as we would if we had more than 2 output layers, using cross-entropy as the loss function.